In [1]:
from pathlib import Path
from dotenv import load_dotenv

dotenv_path = Path("../help-desk-agent-flask-app/.env").resolve()
load_dotenv(dotenv_path=dotenv_path)

True

### Load Document and Vector Store

In [2]:
file_path = Path("../../datasets/help-desk-tickets/va_helpdesk_tickets_sample_realistic.csv").resolve()

In [3]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(
    file_path=file_path, 
    encoding="utf-8",
    metadata_columns=[
        "TicketID", "CreatedAt", "ClosedAt"
    ],
    
)

documents = loader.load()

print("There are {} documents".format(len(documents)))
documents[:5]

There are 500 documents


[Document(metadata={'source': 'C:\\Users\\Tstre\\repos\\datasets\\help-desk-tickets\\va_helpdesk_tickets_sample_realistic.csv', 'row': 0, 'TicketID': 'b44a8a40', 'CreatedAt': '2024-10-12 02:20:35', 'ClosedAt': ''}, page_content='UserID: U008b1810\nUserRole: IT Support\nFacility: San Diego VA\nEHRModule: Authentication\nCategory: Performance Issue\nPriority: Critical\nStatus: Open\nResolutionSummary: \nBody: The Authentication module is taking more than 15 seconds to load patient data. This is slowing down our response time during clinic hours.'),
 Document(metadata={'source': 'C:\\Users\\Tstre\\repos\\datasets\\help-desk-tickets\\va_helpdesk_tickets_sample_realistic.csv', 'row': 1, 'TicketID': '7d9c3369', 'CreatedAt': '2024-05-19 03:44:35', 'ClosedAt': ''}, page_content='UserID: U9aa88516\nUserRole: Lab Technician\nFacility: Miami VA Healthcare\nEHRModule: Pharmacy\nCategory: Bug\nPriority: Low\nStatus: In Progress\nResolutionSummary: \nBody: Since the last update, the Pharmacy interfa

In [2]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Loads the vector store from the specified directory if it already exists
VECTOR_STORE = Chroma(
    collection_name="helpdesk_tickets",
    embedding_function=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
    persist_directory="../../datasets/help-desk-tickets/chroma_db"
)
# Initial run: build and persist
# Uncomment the next line to add new documents and persist the vector store
# _ = VECTOR_STORE.add_documents(documents)

### Develop RAG Tool

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import convert_runnable_to_tool
from pydantic import BaseModel, Field

class InputSchema(BaseModel):
    question: str = Field(
        description=(
            "A natural language question about electronic health record (EHR) system "
            "tickets or issues, such as 'What is causing pharmacy work stoppages?'"
        )
    )

def get_top_k_documents(query: str):
    """Retrieve the top-k documents based on the hyde query."""
    top_k_documents = VECTOR_STORE.similarity_search(query, k=3)
    return top_k_documents

def format_documents(documents):
    """
    Format retrieved docs so that each begins with [TicketID],
    followed by EHR fields like Facility, EHRModule, etc.
    Ensures that TicketID is exposed in exactly bracketed form.
    """
    formatted = []
    for doc in documents:
        tid = doc.metadata.get("TicketID", "UNKNOWN_ID")
        content = doc.page_content
        formatted.append(f"TicketID: [{tid}]\n{content}")
    return "\n\n".join(formatted)

def get_rag_tool():
    """
    Create a RAG tool that uses the GoogleGenerativeAI model to generate responses
    """
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0.7)

    hyde_template = (
    "Based on the question: {question} "
    "Write a passage that could contain the answer to the question: " 
    )
    hyde_prompt = PromptTemplate(
        input_variables=["question"],
        template=hyde_template
    )

    get_top_k_documents_rl = RunnableLambda(get_top_k_documents)
    format_documents_rl = RunnableLambda(format_documents)

    hyde_chain = hyde_prompt | llm | StrOutputParser()
    retreival_chain = get_top_k_documents_rl | format_documents_rl | StrOutputParser()
    rag_chain = hyde_chain | retreival_chain

    rag_tool = convert_runnable_to_tool(
        runnable=rag_chain,
        name="ehr_ticket_rag_retriever",
        description=(
            "Useful for answering questions about EHR system issues using information extracted from support tickets. "
            "Given a question, it generates a hypothesis passage, retrieves relevant tickets, and formats their contents."
        ),
        args_schema=InputSchema
    )
    return rag_tool

In [4]:
from langchain_core.tools import BaseTool
rag_tool = get_rag_tool()
assert isinstance(rag_tool, BaseTool)
print(f"Tool name: {rag_tool.name}")
print(f"Tool name: {rag_tool.description}")
print(f"Tool schema: {rag_tool.args_schema.model_json_schema()}")

Tool name: ehr_ticket_rag_retriever
Tool name: Useful for answering questions about EHR system issues using information extracted from support tickets. Given a question, it generates a hypothesis passage, retrieves relevant tickets, and formats their contents.
Tool schema: {'properties': {'question': {'description': "A natural language question about electronic health record (EHR) system tickets or issues, such as 'What is causing pharmacy work stoppages?'", 'title': 'Question', 'type': 'string'}}, 'required': ['question'], 'title': 'InputSchema', 'type': 'object'}


In [5]:
question = "Can you summarize what is causing some of the pharmacy work stoppage issues?"

response = rag_tool.invoke({"question": question})
print("Response:\n", response)

Response:
 TicketID: [c27b6138]
UserID: Uf61d0731
UserRole: Nurse
Facility: Boston VA
EHRModule: Pharmacy
Category: Performance Issue
Priority: Medium
Status: Open
ResolutionSummary: 
Body: As a Nurse, it’s hard to work when the Pharmacy screen stalls every few clicks. Is there backend maintenance happening?

TicketID: [48cf2892]
UserID: Uc1b5e9b7
UserRole: Nurse
Facility: Denver VA
EHRModule: Pharmacy
Category: Performance Issue
Priority: Low
Status: In Progress
ResolutionSummary: 
Body: As a Nurse, it’s hard to work when the Pharmacy screen stalls every few clicks. Is there backend maintenance happening?

TicketID: [0dd06ea2]
UserID: Ua3f6e391
UserRole: Nurse
Facility: Phoenix VA
EHRModule: Pharmacy
Category: Performance Issue
Priority: Medium
Status: Closed
ResolutionSummary: Resolved per SOP.
Body: As a Nurse, it’s hard to work when the Pharmacy screen stalls every few clicks. Is there backend maintenance happening?


### ReACT Agent

In [8]:
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI

def create_agent():
    """
    Create a ReACT agent that uses the RAG tool for answering questions
    about EHR support tickets.
    """
    # Define the system prompt for the agent
    system_prompt = """
    You are a support assistant for EHR support tickets. Tools retrieve actual ticket records, and Ticket IDs are shown in square brackets — e.g. [c27b6138].

    When answering a user's question:
    - Summarize the explanation using Ticket IDs only in square brackets, never full metadata dump.
    - For example: “The issue appears in [c27b6138], [48cf2892], [0dd06ea2] — it's related to delays in the Pharmacy module…”
    - Always include at least one bracketed TicketID if relevant.
    - If no ticket matches, say “I couldn't find any relevant ticket for that question.”

    - Provide answers that are strictly based on ticket data. Do NOT hallucinate Ticket IDs.
    """
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=1.0)
    rag_tool = get_rag_tool()
    return create_react_agent(model=llm, tools=[rag_tool], state_modifier=system_prompt)

REACT_AGENT = create_agent()

In [9]:
for event in REACT_AGENT.stream({"messages": [("user", question)]}, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Can you summarize what is causing some of the pharmacy work stoppage issues?
================================== Ai Message ==================================
Tool Calls:
  ehr_ticket_rag_retriever (045cb2e4-0777-4896-ae7b-b3f5dd14812c)
 Call ID: 045cb2e4-0777-4896-ae7b-b3f5dd14812c
  Args:
    question: What is causing pharmacy work stoppage issues?
================================= Tool Message =================================
Name: ehr_ticket_rag_retriever

TicketID: [c27b6138]
UserID: Uf61d0731
UserRole: Nurse
Facility: Boston VA
EHRModule: Pharmacy
Category: Performance Issue
Priority: Medium
Status: Open
ResolutionSummary: 
Body: As a Nurse, it’s hard to work when the Pharmacy screen stalls every few clicks. Is there backend maintenance happening?

TicketID: [0dd06ea2]
UserID: Ua3f6e391
UserRole: Nurse
Facility: Phoenix VA
EHRModule: Pharmacy
Category: Performance Issue
Priority: Medium
Status: Clos

In [ ]:
REACT_AGENT.invoke({"messages": [("user", question)]})

{'messages': [HumanMessage(content='Can you summarize what is causing some of the pharmacy work stoppage issues?', additional_kwargs={}, response_metadata={}, id='51310739-24dc-4569-8f02-0ebeb2ca4bdd'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'ehr_ticket_rag_retriever', 'arguments': '{"question": "What is causing pharmacy work stoppage issues?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--9840a481-dc43-46e4-9971-a736ec9cabab-0', tool_calls=[{'name': 'ehr_ticket_rag_retriever', 'args': {'question': 'What is causing pharmacy work stoppage issues?'}, 'id': '4db3c67b-37bb-4214-a61a-4af03a6caeab', 'type': 'tool_call'}], usage_metadata={'input_tokens': 114, 'output_tokens': 27, 'total_tokens': 141, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='TicketID: [c27b6138]\nUserID: Uf61d0731\nUserRole: Nurse\nFacility: Boston VA\nEHRModule: Pharmacy\nCate